# Assignment
You were asked to analyse the last month of bike-sharing data in NYC (data source: https://citibikenyc.com/system-data ), and your manager thinks we could add insurance coverage for **rides that last more than 30 minutes**.

1. How many trips would be covered?

2. If your manager thinks we could charge 0.2 USD for each ride that takes longer than 30 minutes, how much revenue could we expect?

3. Your manager wants to understand the travel distance in distance buckets (0-1,2-4,4-9,10+). Please make a diagram.
 
Please present your analysis in a Jupyter notebook.

In [21]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

In [22]:
from ingest import download_zip_file, extract_zip_archive

# Ingestion
Ingest the February 2024 data from https://citibikenyc.com/system-data and extract the zip archive. The resulting data is stored in the `/raw` folder.

In [23]:
file_name = "202402-citibike-tripdata.csv"
download_base_url = "https://s3.amazonaws.com/tripdata"

zip_archive_path = download_zip_file(file_name, download_base_url)

In [24]:
extract_zip_archive(zip_archive_path)

In [25]:
spark: SparkSession = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("Python Spark SQL basic example") \
        .getOrCreate()

In [26]:
df: DataFrame = spark.read.csv("raw", header=True)

## Explore the data
Check the schema, sample values and display descriptive stats

In [27]:
df.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- ended_at: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: string (nullable = true)
 |-- start_lng: string (nullable = true)
 |-- end_lat: string (nullable = true)
 |-- end_lng: string (nullable = true)
 |-- member_casual: string (nullable = true)



In [28]:
df.sample(fraction=0.1).show()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|        start_lng|          end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+------------------+-------------+
|3AE36736C8C79EBC|electric_bike|2024-02-01 09:00:46|2024-02-01 09:21:20|Court St & Nelson St|         4077.07|Lafayette St & Gr...|       5422.09|      40.67626214|    -73.998769522|         40.72028|         -73.99879|       member|
|1F1466ACBCA41D86| classic_bike|2024-02-07 11:09:38|2024-02-07 1

In [29]:
df.describe().show()

24/03/29 22:24:15 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 13
CSV file: file:///Users/cedricaltermatt/devm/personal/citibikency/raw/202402-citibike-tripdata.csv.zip


+-------+-----------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|          ride_id|       rideable_type|        started_at|            ended_at|  start_station_name|    start_station_id|    end_station_name|      end_station_id|           start_lat|           start_lng|             end_lat|             end_lng|       member_casual|
+-------+-----------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|          2909911|             2408502|           2250089|             2199860|             2182108|             2176971|             2172476|             21

## Prepare data for analysis

In [30]:

staged_df = (
    df.withColumn("start_station_id", df.start_station_id.cast(FloatType()))
    .withColumn("end_station_id", df.end_station_id.cast(FloatType()))
    .withColumn("started_at", F.to_timestamp(df.started_at, "yyyy-MM-dd HH:mm:ss"))
    .withColumn("ended_at", F.to_timestamp(df.ended_at, "yyyy-MM-dd HH:mm:ss"))
    .withColumn("start_lat", df.start_lat.cast(FloatType()))
    .withColumn("start_lng", df.start_lng.cast(FloatType()))
    .withColumn("end_lat", df.end_lat.cast(FloatType()))
    .withColumn("end_lng", df.end_lng.cast(FloatType()))
)

In [31]:
staged_df.show(10)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+---------+----------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat| start_lng|  end_lat|   end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+---------+----------+-------------+
|305C7BE0B5B34D09|electric_bike|2024-02-23 17:02:15|2024-02-23 17:13:21| Murray St & West St|         5329.08|Fulton St & Adams St|       4637.06| 40.71538| -74.01263|40.692417|-73.989494|       member|
|009A4C1BA2E187DB|electric_bike|2024-02-09 08:49:28|2024-02-09 08:53:59|Morris Ave & E 16...|         8036.05|Jerome Ave & Ande...|       8061.01|40.827152| -73.91774|40.830177| -73.92875|

In [32]:
staged_df.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: float (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: float (nullable = true)
 |-- start_lat: float (nullable = true)
 |-- start_lng: float (nullable = true)
 |-- end_lat: float (nullable = true)
 |-- end_lng: float (nullable = true)
 |-- member_casual: string (nullable = true)



# Solve assignment

1. How many trips would be covered?

2. If your manager thinks we could charge 0.2 USD for each ride that takes longer than 30 minutes, how much revenue could we expect?

3. Your manager wants to understand the travel distance in distance buckets (0-1,2-4,4-9,10+). Please make a diagram.